In [1]:
%load_ext autoreload
%autoreload 2

This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
%matplotlib inline
fig_width_pt = 525  # Get this from LaTeX using \showthe\columnwidth
fig_width_pt = 618  # Get this from LaTeX using \showthe\columnwidth
fig_width_pt = 1024  # Get this from LaTeX using \showthe\columnwidth
ppi = 72.27 # (constant) definition of the ppi = points per inch
inches_per_pt = 1.0/ppi  # Convert pt to inches
#inches_per_cm = 1./2.54
figwidth = fig_width_pt*inches_per_pt  # width in inches
phi = (np.sqrt(5) + 1. ) /2 # golden ratio is good for your eyes
dpi_export = 600

In [4]:
figname = '../figures/fig_result'

## Simple Fixation Map training (2 layers)
with clutter / whitening

In [ ]:
verbose = True

In [ ]:
import sys
sys.path.append('../figures')
from retina import MotionCloudNoise, vectorization, get_data_loader, minmax

In [ ]:
N_theta = 6
N_azimuth = 16
N_eccentricity = 10
N_phase = 2

N_pic = 128
N_X = N_pic
N_Y = N_pic
rho = 1.41
verbose = 1


In [ ]:
retina_transform = vectorization(N_theta, N_azimuth, N_eccentricity, N_phase, N_X, N_Y, rho)

In [ ]:
# retina_vector = retina.reshape((N_theta*N_azimuth*N_eccentricity*N_phase, N_X*N_Y))
# retina_inverse = np.linalg.pinv(retina_vector)

In [ ]:
loader = get_data_loader(batch_size=100, train = False)

In [ ]:
path = "../data/MNIST_accuracy.npy"
import os
if os.path.isfile(path):
    accuracy_map =  np.load(path)
    if verbose:
        print('Loading accuracy... min, max=', accuracy_map.min(), accuracy_map.max())
else:
    print('No accuracy data found.')

In [ ]:
plt.figure(figsize = (10,3))
plt.subplot(121)
plt.imshow(accuracy_map)
plt.subplot(122)
_ = plt.plot(accuracy_map)

In [ ]:
from retina import do_offset, accuracy_fullfield

In [ ]:
colliculus = (retina_transform**2).sum(axis=(0, 3))
#colliculus = colliculus**.5
colliculus /= colliculus.sum(axis=-1)[:, :, None]
print(colliculus.shape)

In [ ]:
colliculus_vector = colliculus.reshape((N_azimuth*N_eccentricity, N_X*N_Y))
print(colliculus_vector.shape)

In [ ]:
colliculus_inverse = np.linalg.pinv(colliculus_vector)
print(colliculus_inverse.shape)

### From MNIST encoding

In [ ]:
from retina import place_object

# Hyperparameters

In [ ]:
minibatch_size = 100  # quantity of examples that'll be processed
lr = 1e-4 #1e-3  #0.05

OFFSET_STD = 15 #
OFFSET_MAX = 30 #
NOISE = 1 #0 #
CONTRAST = 0.5 #1 #
sf_0 = 0.2
B_sf = 0.3

### Test

In [ ]:
from retina import retina, retina_inverse
retina_inverse_transform = retina_inverse(retina_transform)

In [ ]:
data, label = next(iter(loader))
fig, axs = plt.subplots(1, 2, figsize = (figwidth, figwidth/2))
i = 4
offset_std=OFFSET_STD
offset_max=OFFSET_MAX
i_offset = minmax(np.random.randn() * offset_std, offset_max)
j_offset = minmax(np.random.randn() * offset_std, offset_max)

data_fullfield = place_object(data[i, 0, :, :].numpy(), i_offset, j_offset, 
                                CONTRAST=CONTRAST, NOISE=NOISE,
                                sf_0=sf_0, B_sf=B_sf)
data_retina, tensor_retina  =  retina(data_fullfield, retina_transform)

axs[0].imshow(data_fullfield, cmap=plt.gray(), vmin=0, vmax=1)
axs[1].imshow((retina_inverse_transform @ data_retina).reshape(N_pic, N_pic), cmap=plt.viridis())# vmin=0, vmax=1, 


#### Orientation invariant power encoding (colliculus??)

In [ ]:
# accuracy_fullfield_map = do_offset(accuracy_map, i_offset, j_offset, N_pic=N_pic, min=0.1)
# accuracy_colliculus = colliculus_vector @ accuracy_fullfield_map.ravel()
accuracy_colliculus, accuracy_fullfield_map = accuracy_fullfield(accuracy_map, i_offset, j_offset, N_pic, colliculus_vector)
im = colliculus_inverse @ accuracy_colliculus

fig, axs = plt.subplots(1, 2, figsize = (figwidth, figwidth/2))
axs[0].imshow(accuracy_fullfield_map, vmin=0, vmax=1, cmap=plt.viridis())
axs[1].imshow(im.reshape(N_pic, N_pic), vmin=0, vmax=1, cmap=plt.viridis())



#### Torch stuff

In [ ]:
import torch
do_cuda = False # torch.cuda.is_available()
kwargs = {'num_workers': 4, 'pin_memory': True} if do_cuda else {}
device = torch.device("cuda" if do_cuda else "cpu")

In [ ]:
train_loader = get_data_loader(batch_size=minibatch_size, train = True)
test_loader = get_data_loader(batch_size=1000, train = False)

# Network

In [ ]:
#FIC_NAME = "2019-02-15-anywhere-additive-noise-white-1000-Necc-10.npy"
# FIC_NAME = "../data/2019-03-02-anywhere-background-noise-white-1000.npy"
# FIC_NAME = "../data/2019-03-07_CNS.npy"
# FIC_NAME = "../data/2019-03-08_CNS.npy"
FIC_NAME = "../data/2019-03-11_CNS.npy"

In [ ]:
BIAS_CONV = True
BIAS_DECONV = True #True

class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        #self.bn1= torch.nn.Linear(N_theta*N_azimuth*N_eccentricity*N_phase, 200, bias = BIAS_DECONV)
        self.bn1= torch.nn.Linear(N_theta*N_azimuth*N_eccentricity*N_phase, 1000, bias = BIAS_DECONV)
        #self.bn2 = torch.nn.Linear(200, 80, bias = BIAS_DECONV)
        self.bn2 = torch.nn.Linear(1000, 1000, bias = BIAS_DECONV)
        #self.bn3 = torch.nn.Linear(80, N_azimuth*N_eccentricity, bias = BIAS_DECONV)
        self.bn3 = torch.nn.Linear(1000, N_azimuth*N_eccentricity, bias = BIAS_DECONV)
                
    def forward(self, image):
       
        h_bn1 = F.relu(self.bn1(image))               
        h_bn2 = F.relu(self.bn2(h_bn1))
        h_bn2_drop = F.dropout(h_bn2, p = .5) 
        u = self.bn3(h_bn2_drop)
        
        return u


In [ ]:
net = Net()

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
loss_func = torch.nn.BCEWithLogitsLoss() #torch.nn.CrossEntropyLoss()

#### Training

In [ ]:
import time
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
def train(net, minibatch_size,
          optimizer=optimizer,
          vsize = N_theta * N_azimuth * N_eccentricity * N_phase,
          asize = 1,
          offset_std=OFFSET_STD,
          offset_max=OFFSET_MAX,
          verbose=1,
          CONTRAST=CONTRAST,
          NOISE=NOISE,
          sf_0=sf_0, 
          B_sf=B_sf):
    
    t_start = time.time()
    
    if verbose: print('Starting training...')
    
    for batch_idx, (data, label) in enumerate(train_loader):
        optimizer.zero_grad()

        retina_data = np.zeros((minibatch_size, N_phase * N_theta * N_azimuth * N_eccentricity))
        fixmap_data = np.zeros((minibatch_size, N_azimuth * N_eccentricity))

        for i in range(minibatch_size):
            i_offset = minmax(np.random.randn() * offset_std, offset_max)
            j_offset = minmax(np.random.randn() * offset_std, offset_max)
            
            data_fullfield = place_object(data[i, 0, :, :].numpy(), i_offset, j_offset, 
                                            CONTRAST=CONTRAST, NOISE=NOISE,
                                            sf_0=sf_0, B_sf=B_sf)
            retina_data[i, :], _  =  retina(data_fullfield, retina_transform)

            fixmap_data[i,:], _ = accuracy_fullfield(accuracy_map, i_offset, j_offset, N_pic, colliculus_vector)
            
        retina_data = Variable(torch.FloatTensor(retina_data))
        fixmap_data = Variable(torch.FloatTensor(fixmap_data))
        
        prediction = net(retina_data)
        loss = loss_func(prediction, fixmap_data)
        loss.backward()
        optimizer.step()

        if verbose and batch_idx % 10 == 0:
            acc, _ = accuracy_gain(prediction, fixmap_data, minibatch_size)
            print('[%d/%d] Loss: %.3f Acc : %.3f'%(batch_idx*minibatch_size, len(train_loader.dataset),loss.data.numpy(), acc))
            f = open(FIC_NAME.replace('npy', 'txt'), 'a')
            f.write('%.5f\t%.5f'%(loss, acc))    
            f.close()
    return net


In [ ]:
def test(net, optimizer=optimizer,
         vsize=N_theta*N_azimuth*N_eccentricity*N_phase,
         asize=N_azimuth*N_eccentricity, offset_std=OFFSET_STD, offset_max=OFFSET_MAX, 
         CONTRAST=CONTRAST, NOISE=NOISE,
         sf_0=sf_0, 
         B_sf=B_sf):
    
    #for batch_idx, (data, label) in enumerate(test_loader):
    data, label = next(iter(test_loader))
    batch_size = label.shape[0]

    retina_data = np.zeros((batch_size, N_phase * N_theta * N_azimuth * N_eccentricity))
    fixmap_data = np.zeros((batch_size, N_azimuth * N_eccentricity))

    for i in range(batch_size):
        i_offset = minmax(np.random.randn() * offset_std, offset_max)
        j_offset = minmax(np.random.randn() * offset_std, offset_max)
        data_fullfield = place_object(data[i, 0, :, :].numpy(), i_offset, j_offset, 
                                        CONTRAST=CONTRAST, NOISE=NOISE,
                                        sf_0=sf_0, B_sf=B_sf)
        retina_data[i, :], _  =  retina(data_fullfield, retina_transform)

        #accuracy_fullfield = do_offset(accuracy_map, i_offset, j_offset, N_pic=128, min=0.1)
        fixmap_data[i,:], _ = accuracy_fullfield(accuracy_map, i_offset, j_offset, N_pic, colliculus_vector)
        #colliculus_vector @ accuracy_fullfield.ravel()

    retina_data = Variable(torch.FloatTensor(retina_data))
    fixmap_data = Variable(torch.FloatTensor(fixmap_data))
    
    with torch.no_grad():
        output = net(retina_data) #.data.numpy()
        #indices_max = np.zeros(batch_size, dtype = 'int')
        #for i in range(batch_size):
        #    indices_max[i] = np.where(output[i,:] == max(output[i,:]))[0][0]
        #acc = np.mean(fixmap_data.data.numpy()[:,indices_max])
        #acc = 0.1 + (acc - 0.1) * 2.5
        acc, _ = accuracy_gain(output, fixmap_data, batch_size)

    return acc


## Anywhere target, with noise

#### Additive noise + whitening

In [ ]:
if not os.path.isfile(FIC_NAME):
    for epoch in range(40):
        train(net, minibatch_size)
        loss = test(net)
        print('Test set: Final Loss: %.3f'%loss) 
    torch.save(net, FIC_NAME)
else:
    net = torch.load(FIC_NAME)       

# Visualisation

In [ ]:
vsize = N_theta * N_azimuth * N_eccentricity * N_phase
asize = N_azimuth * N_eccentricity
offset_std=OFFSET_STD
offset_max=OFFSET_MAX

test_batch_size = 50
test_loader = get_data_loader(batch_size=test_batch_size, train = False)

data, label = next(iter(test_loader))

In [ ]:
input_n = np.zeros((test_batch_size, 1, vsize))
a_data_n = np.zeros((test_batch_size, 1, asize))
full_data = np.zeros((test_batch_size, 128, 128))
full_fixmap_n = np.zeros((test_batch_size, 128, 128))
        # target = np.zeros((minibatch_size, asize))

for idx in range(test_batch_size):
    i_offset = minmax(np.random.randn() * offset_std, offset_max)
    j_offset = minmax(np.random.randn() * offset_std, offset_max)
    
    data_fullfield = place_object(data[idx, 0, :, :].numpy(), i_offset, j_offset, 
                                    CONTRAST=CONTRAST, NOISE=NOISE,
                                    sf_0=sf_0, B_sf=B_sf)
    input_n[idx, 0, :], _  =  retina(data_fullfield, retina_transform)

    a_data_n[idx, 0, :], _ = accuracy_fullfield(accuracy_map, i_offset, j_offset, N_pic, colliculus_vector)
    #full_fixmap_n[idx, :, :] = 

In [ ]:
delta = 1/N_azimuth
log_r, theta = np.meshgrid(np.linspace(0, 1, N_eccentricity + 1), np.linspace(-np.pi*(.5 + delta), np.pi*(1.5 - delta), N_azimuth + 1))
#for idx in range(test_batch_size):
for idx in range(30, 35):
    plt.figure(figsize = (15, 8))
    plt.subplot(141)
    plt.imshow(data[idx, 0, :, :].numpy())

    plt.subplot(142)
    im = retina_inverse_transform @ input_n[idx,0,:]
    #plt.plot(input_n[idx,0,:])
    #plt.subplot(162)
    plt.imshow(im.reshape(128, 128))
    plt.plot(63.5, 63.5, 'r+')
    plt.title(idx)
    #plt.subplot(132)
    #plt.plot(a_data_n[idx,0,:])    
    col = colliculus_inverse @ a_data_n[idx,0,:]
    ax = plt.subplot(143, projection='polar')
    vec_t = Variable(torch.FloatTensor(a_data_n[idx,0,:]))
    vec_t = vec_t.reshape((1,N_azimuth * N_eccentricity))
    ax.pcolor(theta, log_r, vec_t.reshape((N_azimuth, N_eccentricity)))
    #plt.imshow(col.reshape(128, 128))
    in_t = Variable(torch.FloatTensor(input_n[idx,0,:]))
    out_t = net(in_t)
    out_sig = F.sigmoid(out_t).detach().numpy()
    out_t = out_t.reshape((1,N_azimuth * N_eccentricity))
    #acc, _ = accuracy_gain(out_t.reshape(1,N_azimuth * N_eccentricity), vec_t.reshape(1,N_azimuth * N_eccentricity), 1)
    acc, _ = accuracy_gain(out_t, vec_t, 1, full_fixmap = full_fixmap_n[idx, :].reshape((1, -1)))
    #plt.subplot(165)
    #plt.plot(out_sig)    
    #plt.title(acc)
    view = colliculus_inverse @ out_sig.flatten()
    ax = plt.subplot(144, projection='polar')
    ax.pcolor(theta, log_r, out_sig.reshape((N_azimuth, N_eccentricity)))
    #plt.imshow(view.reshape(128, 128))
                                 

In [ ]:
if False:
    i = 0
    offset_std=OFFSET_STD
    offset_max=OFFSET_MAX
    i_offset = minmax(np.random.randn() * offset_std, offset_max)
    j_offset = minmax(np.random.randn() * offset_std, offset_max)
            
    data_fullfield = place_object(data[i, 0, :, :].numpy(), i_offset, j_offset, 
                                    CONTRAST=CONTRAST, NOISE=NOISE,
                                    sf_0=sf_0, B_sf=B_sf)
    input_vector, _  =  retina(data_fullfield, retina_transform)

    coll_fixmap, _ = accuracy_fullfield(accuracy_map, i_offset, j_offset, N_pic, colliculus_vector)

    #plt.imshow(input_test)
    in_ = Variable(torch.FloatTensor(input_vector))
    out = net(in_)
    
    plt.figure(figsize = (20,6))

    plt.subplot(151)
    plt.imshow(image)

    plt.subplot(152)
    f = plt.plot(coll_fixmap)
    f = plt.plot(F.sigmoid(out.data))

    plt.subplot(153)
    im = colliculus_inverse @ coll_fixmap
    plt.imshow(im.reshape(128, 128))
    print(max(coll_fixmap))
    print(max(im))

    plt.subplot(154)
    im_pred = colliculus_inverse @ F.sigmoid(out).data.numpy().flatten()
    plt.imshow(im_pred.reshape(128, 128))
    ind_pred = np.where(im_pred == max(im_pred))
    print(im[ind_pred])

    plt.subplot(155)
    test = F.sigmoid(out).data.numpy().reshape((N_azimuth, N_eccentricity))
    indices_ij = np.where(test == max(test.flatten()))
    azimuth = indices_ij[0][0]
    eccentricity = indices_ij[1][0]
    full_masque = colliculus[azimuth,eccentricity,:].reshape(128, 128)
    plt.imshow(im_pred.reshape(128, 128) * full_masque)
    


In [ ]:
def accuracy_gain(prediction, fixmap_data, batch_size, full_fixmap=None):
    """
    Actuate the predictions of the ``where'' pathway to compute final accuracy using the ``what'' pathway
    
    """
    acc = []
    
    for i in range(batch_size):
        fixmap_coll =  fixmap_data[i, :].data.numpy()
        pred_coll = F.sigmoid(prediction[i, :]).data.numpy()
        
        indice_max_coll = np.where(pred_coll == max(pred_coll))[0][0]
        acc_coll = fixmap_coll[indice_max_coll]
                
        test = pred_coll.reshape((N_azimuth, N_eccentricity))
        indices_ij = np.where(test == max(test.flatten()))
        azimuth = indices_ij[0][0]
        eccentricity = indices_ij[1][0]
        full_masque = colliculus[azimuth,eccentricity,:] #> 0.0003
        
        if full_fixmap is not None:
            full_ref = full_fixmap[i,:]
            #print('OK')
        else:
            full_ref = colliculus_inverse @ fixmap_coll.flatten()
            
        full_pred = colliculus_inverse @ pred_coll.flatten()
        #full_masque = colliculus_inverse @ masque.flatten()
        #masque[np.where(masque < 0.11)] = 0
#        full_pred *= full_masque
        full_pred = full_masque
        #full_pred = colliculus_inverse @ pred_coll_test.flatten()
        
        indice_max_full = np.where(full_pred == max(full_pred))[0][0]
        #print(indice_max_full)
        acc_full = full_ref[indice_max_full] 
                
        #acc[i] = max(acc_full, acc_coll)
        if acc_full > acc_coll:
            acc += [acc_full]
            
        #acc[i] = 0.1 + (acc[i] - 0.1) * 2.25
    #print(indices_max)
    if len(acc) > 0:
        acc_mean = np.mean(acc) #fixmap_data.data.numpy()[:,indices_max])
    else:
        acc_mean = acc_coll
    #acc = np.mean(fixmap_data[:,indices_max])
    return acc_mean, acc

In [ ]:
vsize = N_theta * N_azimuth * N_eccentricity * N_phase
asize = N_azimuth * N_eccentricity
offset_std=OFFSET_STD
offset_max=OFFSET_MAX

N_pic_mnist = 28
N_class_mnist = 10

test_batch_size = 1000
N_eccentricities = 9
mem_acc_log = []
mem_acc_data_log = []
mem_ref_log = [] 
mem_ref_data_log = []

ecc_max=.8
r_pix = []

eccentricities = range(N_eccentricities)
for i_eccentricity in eccentricities:
    
    eccentricity = ecc_max * (1/rho)**(N_eccentricity - i_eccentricity)
    
    r = np.sqrt(N_X**2+N_Y**2) / 2 * eccentricity
    print('at scale ', i_eccentricity, ' r=', r)
    r_pix.append(r)

    test_loader_2 = get_data_loader(batch_size=test_batch_size, train = False)

    data, label = next(iter(test_loader_2))
    input_n = np.zeros((test_batch_size, 1, vsize))
    a_data = np.zeros((test_batch_size, asize))
    full_fixmap_n = np.zeros((test_batch_size, 128 * 128))
        
    ref_data = np.zeros(test_batch_size)

    for idx in range(test_batch_size):
        theta = np.random.rand() * 2 * np.pi
        i_offset = int(r * np.cos(theta))
        j_offset = int(r * np.sin(theta))
            
        # changed
        data_fullfield = place_object(data[idx, 0, :, :].numpy(), i_offset, j_offset, 
                                        CONTRAST=CONTRAST, NOISE=NOISE,
                                        sf_0=sf_0, B_sf=B_sf)
        input_n[idx, 0, :], _  =  retina(data_fullfield, retina_transform)

        a_data[idx, :], accuracy_fullfield_map = accuracy_fullfield(accuracy_map, i_offset, j_offset, N_pic, colliculus_vector)
        full_fixmap_n[idx,:] = accuracy_fullfield_map.flatten()
        
        # when we do not do a saccade, the reference accurracy data is pre-computed 
        if r <= N_pic_mnist:
            ref_data[idx] = accuracy_map[N_pic_mnist - 1 + i_offset, N_pic_mnist - 1 + j_offset]
        else:
            ref_data[idx] = 1 / N_class_mnist      
        
        
    in_t = Variable(torch.FloatTensor(input_n))
    out_t = net(in_t)
    a_data_t = Variable(torch.FloatTensor(a_data))
    acc, acc_data = accuracy_gain(out_t, a_data_t, test_batch_size, full_fixmap=full_fixmap_n)
    mem_acc_log += [acc]
    mem_acc_data_log += [acc_data]
    
    mem_ref_log += [np.mean(ref_data)]
    

In [ ]:
np.max(acc_data)

In [ ]:
theta, log_r, vec_t

In [ ]:
idx = 30
w = 28
log_r, theta = np.meshgrid(np.linspace(0, 1, N_eccentricity + 1), np.linspace(-np.pi*(.5 + delta), np.pi*(1.5 - delta), N_azimuth + 1))

fig = plt.figure(figsize = (figwidth, figwidth/2.5))#1.618))#, constrained_layout=True)
ax_A = plt.subplot(1, 4, 1)
im = retina_inverse_transform @ input_n[idx, 0, :]
ax_A.imshow(im.reshape(128, 128))
ax_A.plot(64.5, 64.5, 'r+', markersize = 14, mew=3)
ax_A.axis('off')
#ax_A.set_title('Input', fontsize = 14)

ax_B = plt.subplot(2, 4, 2, projection='polar', autoscale_on=False)
vec_t = Variable(torch.FloatTensor(a_data_n[idx, 0, :]))
vec_t = vec_t.reshape((1,N_azimuth * N_eccentricity))
ax_B.pcolor(theta, log_r, vec_t.reshape((N_azimuth, N_eccentricity)))
ax_B.grid('off')
#ax.set_rgrids('off')
plt.title('True', fontsize = 14)
ax_B.set_yticklabels([])
ax_B.set_xticklabels([])

ax_Bb = plt.subplot(2, 4, 6, projection='polar')
in_t = Variable(torch.FloatTensor(input_n[idx,0,:]))
out_t = net(in_t)
out_sig = F.sigmoid(out_t).detach().numpy()
ax_Bb.pcolor(theta, log_r, out_sig.reshape((N_azimuth, N_eccentricity)))
ax_Bb.set_title('Predicted', fontsize = 14)
ax_Bb.set_yticklabels([])
ax_Bb.set_xticklabels([])

test = out_sig.reshape((N_azimuth, N_eccentricity))
indices_ij = np.where(test == max(test.flatten()))
azimuth = indices_ij[0][0]
eccentricity = indices_ij[1][0]
full_masque = colliculus[azimuth,eccentricity,:]
indice_move = np.where(full_masque == max(full_masque))
i_move = indice_move[0][0] // 128 - 64
j_move = indice_move[0][0] % 128 - 64
print(i_move, j_move)

ax_C = plt.subplot(1, 4, 3)
data_fullfield = place_object(data[idx, 0, :, :].numpy(), 0, 0, 
                                CONTRAST=CONTRAST, NOISE=NOISE,
                                sf_0=sf_0, B_sf=B_sf)
input_vector, _  =  retina(data_fullfield, retina_transform)
im = retina_inverse_transform @ input_vector # input_n[idx,0,:]
ax_C.imshow(im.reshape(128, 128))
ax_C.plot(64.5, 64.5, 'r+', markersize = 14, mew=3)
ax_A.arrow(64.5, 64.5, j_move, i_move, width=.3, color='r', head_width=4., length_includes_head=True, edgecolor='k')
for ax in [ax_A, ax_C]:
    ax.plot([N_pic//2-w/2, N_pic//2+w/2, N_pic//2+w/2, N_pic//2-w/2, N_pic//2-w/2], 
            [N_pic//2-w/2, N_pic//2-w/2, N_pic//2+w/2, N_pic//2+w/2, N_pic//2-w/2], '--', color='r', lw=.75, markeredgewidth=1)
plt.axis('off')
vec_t = vec_t.reshape((1,N_azimuth * N_eccentricity))
out_t = out_t.reshape((1,N_azimuth * N_eccentricity))
acc, _ = accuracy_gain(out_t, vec_t, 1, full_fixmap = full_fixmap_n[idx, :].reshape((1, -1)))
#ax_C.set_title('Final accuracy : ' + str(acc), fontsize = 14)

ax_D = plt.subplot(1, 4, 4)
r_ref = r_pix
r_ref = range(9)

ax_D.bar(r_ref, mem_acc_log, alpha = .5, label = 'One saccade')
ax_D.bar(r_ref, mem_ref_log, alpha = .5, label = 'No saccade') #accuracy_map[27,27:55])
ax_D.plot(np.arange(10)-.5, [0.1]*10, ':', c='k', label = 'Baseline')
plt.legend(loc='best')
ax_D.set_title('Class accuracy', fontsize = 14)
ax_D.set_xlabel('Target eccentricity (pixels)', fontsize = 12)
ax_D.set_xticks(range(9))
ax_D.set_xticklabels(['%.1f' % d for  d in r_pix])

ax_D.set_ylim([0,1])

for ax, text in [[ax_A, 'DIS'], [ax_C, 'SAC']]:
    ax.text(4, 15, text, fontsize=24,
          bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
          ha='left', va='center') 

offset = -.015
# y_offset = 1.2
for ax, text, x_offset, y_offset in [[ax_A, 'A', offset, 1.15], [ax_B, 'B', -.25, 1.225], [ax_C, 'C', offset, 1.15], [ax_D, 'D', offset, 1.15]]:
    ax.text(x_offset, y_offset, '(' + text + ')', fontsize=24,
              bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
              ha='left', va='center', transform=ax.transAxes) 

# pos : [left, bottom, width, height] =    The new position of the in `.Figure` coordinates.    
plt.tight_layout()
ax_A.set_position([0.025, 0.1, .3, .45])
ax_B.set_position( [0.24, 0.375, .2, 0.2])
ax_Bb.set_position([0.24, 0.1, .2, 0.2])
ax_C.set_position([0.35, .1, .3, .45])
ax_D.set_position([0.65, .1, .3, .45])
fig.savefig(figname + '.pdf', bbox_inches='tight', pad_inches=0.1)

# post-processing

In [ ]:
!convert  -density {dpi_export} {figname}.pdf {figname}.jpg
!convert  -density {dpi_export} {figname}.pdf {figname}.png
#!convert  -density {dpi_export} -resize 5400  -units pixelsperinch -flatten  -compress lzw  -depth 8 {fname}.pdf {fname}.tiff

In [ ]:
from IPython.display import Image
Image('{figname}.png'.format(figname=figname))

In [ ]:
!ls  -l {figname}*

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts